#### Power Iteration Method
``` txt
    幂迭代法(Power Iteration Method)是一种寻找矩阵最大本征值及其对应本征向量的简单方法。其核心思想是，对一个随机向量反复左乘一个矩阵 A，
经过足够多次迭代后，向量的方向将收敛于 A 的主导本征向量。
```
* [Power_iteration wikipedia](https://en.wikipedia.org/wiki/Power_iteration)

In [5]:
import numpy as np


def power_iteration(A, max_iter=100, tol=1e-6):
    # 获取矩阵大小
    n = A.shape[0]
    
    # 随机初始向量
    u = np.random.rand(n)
    u = u / np.linalg.norm(u)
    
    calc_err_v0 = lambda u_new, u : np.linalg.norm(u_new - u)
    # v1 版本比 v0 好的地方在于, 可以处理 u_new 和 u 方向相反的情况
    calc_err_v1 = lambda u_new, u : 1- abs(np.dot(u_new, u))

    for i in range(max_iter):
        Au = A @ u
        u_new = Au / np.linalg.norm(Au)
        
        err = calc_err_v1(u_new, u)

        if err < tol or i == max_iter - 1:
            print(f"Converged at iteration {i}, error={err}")
            break
        
        u = u_new
    
    # 瑞利商计算特征值
    eigenvalue = (u.T @ A @ u) / (u.T @ u)
    return eigenvalue, u

def find_top_eigenvalues(A, num_eigenvalues=2, max_iter=100, tol=1e-6):
    """
    找到前几个最大的特征值和对应的特征向量
    
    参数:
    A: 输入矩阵
    num_eigenvalues: 需要找到的特征值数量
    max_iter: 最大迭代次数
    tol: 收敛容差
    
    返回:
    eigenvalues: 特征值列表（从大到小）
    eigenvectors: 对应的特征向量列表
    """
    n = A.shape[0]
    eigenvalues = []
    eigenvectors = []
    A_current = A.copy()
    
    for i in range(num_eigenvalues):
        print(f"Finding eigenvalue {i+1}...")
        
        # 使用幂迭代法找到当前最大特征值
        eigenvalue, eigenvector = power_iteration(A_current, max_iter, tol)
        eigenvalues.append(eigenvalue)
        eigenvectors.append(eigenvector)
        
        # 如果是最后一个特征值，不需要继续收缩
        if i == num_eigenvalues - 1:
            break
            
        # 使用收缩技术：A = A - λ * v * v^T
        # 这会将当前特征值对应的分量从矩阵中移除
        outer_product = np.outer(eigenvector, eigenvector)
        A_current = A_current - eigenvalue * outer_product
        
        print(f"Found eigenvalue {i+1}: {eigenvalue}")
    
    return eigenvalues, eigenvectors


def find_eigenvalue_close_to_sigma(A, sigma=0, max_iter=100, tol=1e-6):
    """
    逆幂法求最接近sigma的特征值
    
    参数:
    A: 输入矩阵 (n x n)
    sigma: 目标特征值的估计值（移位量）
    max_iter: 最大迭代次数
    tol: 收敛容差
    
    返回:
    eigenvalue: 最接近sigma的特征值
    eigenvector: 对应的特征向量
    """
    n = A.shape[0]
    
    # 构造移位矩阵: (A - σI)
    shifted_A = A - sigma * np.eye(n)
    
    try:
        # 计算移位矩阵的逆
        inv_shifted_A = np.linalg.inv(shifted_A)
    except np.linalg.LinAlgError:
        raise ValueError("Matrix (A - σI) is singular, choose a different sigma")
    
    # 对逆矩阵使用幂迭代法求最大特征值
    # 注意：inv_shifted_A 的最大特征值对应 A 的最接近sigma的特征值
    inv_eigenvalue, eigenvector = power_iteration(inv_shifted_A, max_iter, tol)
    
    # 转换回原矩阵的特征值
    # 如果 μ 是 inv_shifted_A 的特征值，那么 1/μ 是 shifted_A 的特征值
    # 所以 A 的特征值为：λ = σ + 1/μ
    eigenvalue = sigma + 1.0 / inv_eigenvalue
    
    return eigenvalue, eigenvector



In [6]:
# 创建一个对称矩阵（确保实数特征值）
np.random.seed(42)
n = 5
B = np.random.rand(n, n)
A = B + B.T  # 使矩阵对称

print("Original matrix A:")
print(A)

# 找到前两个特征值
eigenvalues, eigenvectors = find_top_eigenvalues(A, num_eigenvalues=2)

print("\nResults:")
print(f"Largest eigenvalue: {eigenvalues[0]}")
print(f"Second largest eigenvalue: {eigenvalues[1]}")

# 验证结果（使用numpy的eig函数）
eigvals, eigvecs = np.linalg.eig(A)
sorted_indices = np.argsort(np.abs(eigvals))[::-1]  # 按模长从大到小排序
sorted_eigvals = eigvals[sorted_indices]

print("\nValidation with numpy.linalg.eig:")
print(f"Largest eigenvalue (numpy): {sorted_eigvals[0]}")
print(f"Second largest eigenvalue (numpy): {sorted_eigvals[1]}")


Original matrix A:
[[0.74908024 1.10670883 0.75257844 0.78206299 0.76787154]
 [1.10670883 0.11616722 1.836086   0.90535725 0.84756644]
 [0.75257844 1.836086   1.66488528 0.73709554 0.47396962]
 [0.78206299 0.90535725 0.73709554 0.86389004 0.65759098]
 [0.76787154 0.84756644 0.47396962 0.65759098 0.91213997]]
Finding eigenvalue 1...
Converged at iteration 5, error=3.0967639486068066e-07
Found eigenvalue 1: 4.528957572646465
Finding eigenvalue 2...
Converged at iteration 25, error=5.476318717390072e-07

Results:
Largest eigenvalue: 4.528957572646465
Second largest eigenvalue: -1.3140917618429957

Validation with numpy.linalg.eig:
Largest eigenvalue (numpy): 4.528959747929156
Second largest eigenvalue (numpy): -1.3140851205023023


In [ ]:
eigenvalues, eigenvectors = find_eigenvalue_close_to_sigma(A, sigma=-1.3)
print(f"searched eigenvalues={eigenvalues}")

Converged at iteration 2, error=1.752098743867947e-07
searched eigenvalues=-1.3140851253930066, eigenvectors=[ 0.22028203 -0.86027191  0.43033994  0.09312722  0.13244973]


```txt
    如果 power ieration 还有不明白的地方, 可以了解一下前置知识 Gram-Schmidt 正交分解，
在有了 power iteration 基础后，可以了解一下 Krylov 子空间迭代法和 Lanczos 方法。
```